In [15]:
import re
import string
import random
import nltk
from nltk.probability import ConditionalFreqDist
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [8]:
def main():
    file = open('alice.txt', 'r')
    text = ""
    while True:
        line = file.readline()
        text += line
        if not line:
            break

    # pre-process text
    print("Filtering...")
    words = filter(text)
    print("Cleaning...")
    words = clean(words)

    # make language model
    print("Making model...")
    model = n_gram_model(words)

    print("Enter a phrase: ")
    user_input = input()
    predict(model, user_input)

In [9]:
def filter(text):
    # normalize text
    text = (unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore'))
    # replace html chars with ' '
    text = re.sub('<.*?>', ' ', text)
    # remove punctuation
    text = text.translate(str.maketrans(' ', ' ', string.punctuation))
    # only alphabets and numerics
    text = re.sub('[^a-zA-Z]', ' ', text)
    # replace newline with space
    text = re.sub("\n", " ", text)
    # lower case
    text = text.lower()
    # split and join the words
    text = ' '.join(text.split())

    return text

## **Tokenize remaining words and perform lemmatization**

In [10]:
def clean(text):
    tokens = nltk.word_tokenize(text)
    wnl = nltk.stem.WordNetLemmatizer()

    output = []
    for words in tokens:
        # lemmatize words
        output.append(wnl.lemmatize(words))

    return output

### **Make a language model using a dictionary, trigrams, and calculate word probabilities**

In [11]:
def n_gram_model(text):
    trigrams = list(nltk.ngrams(text, 3, pad_left=True, pad_right=True, left_pad_symbol='<s>', right_pad_symbol='</s>'))
    # make conditional frequencies dictionary
    cfdist = ConditionalFreqDist()
    for w1, w2, w3 in trigrams:
        cfdist[(w1, w2)][w3] += 1

    # transform frequencies to probabilities
    for w1_w2 in cfdist:
        total_count = float(sum(cfdist[w1_w2].values()))
        for w3 in cfdist[w1_w2]:
            cfdist[w1_w2][w3] /= total_count

    return cfdist

Generate predictions from the Conditional Frequency Distribution dictionary (param: model), append weighted random choice to input phrase, and allow option to generate more words following the prediction

In [16]:
def predict(model, user_input):
    user_input = filter(user_input)
    user_input = user_input.split()

    w1 = len(user_input) - 2
    w2 = len(user_input)
    prev_words = user_input[w1:w2]

    # display prediction from highest to lowest maximum likelihood
    prediction = sorted(dict(model[prev_words[0], prev_words[1]]), key=lambda x: dict(model[prev_words[0], prev_words[1]])[x], reverse=True)
    print("Trigram model predictions: ", prediction)

    word = []
    weight = []
    for key, prob in dict(model[prev_words[0], prev_words[1]]).items():
        word.append(key)
        weight.append(prob)
    # picking from a weighted random probability of predictions
    next_word = random.choices(word, weights=weight, k=1)
    # add predicted word output to our input
    user_input.append(next_word[0])
    print(' '.join(user_input))

    ask = input("Do you want to generate another word? (type 'y' for yes or 'n' for no): ")
    if ask.lower() == 'y':
        predict(model, str(user_input))
    elif ask.lower() == 'n':
        print("done")
        

main()

Filtering...
Cleaning...
Making model...
Enter a phrase: 
alice said to the
Trigram model predictions:  ['other', 'jury', 'door', 'table', 'knave', 'mock', 'gryphon', 'little', 'end', 'shore', 'beginning', 'dormouse', 'game', 'queen', 'garden', 'seaside', 'general', 'cur', 'fifth', 'puppy', 'law', 'baby', 'three', 'king', 'rosetree', 'conclusion', 'cheshire', 'duchess', 'executioner', 'croquetground', 'company', 'classic', 'whiting', 'dance', 'porpoise', 'part', 'caterpillar', 'hatter', 'head', 'tart', 'waving', 'voice', 'confused']
alice said to the gryphon
Do you want to generate another word? (type 'y' for yes or 'n' for no): y
Trigram model predictions:  ['and', 'went', 'it', 'said', 'i', 'you', 'the', 'replied', 'alice', 'sat', 'half', 'answered', 'she', 'before', 'lifted', 'never', 'sighing', 'remarked', 'interrupted', 'of', 'then', 'with', 'turn', 'at', 'we', 'ive', 'that', 'do', 'added', 'in', 'a', 'how', 'well', 'hastily', 'repeated', 'only', 'they', 'whispered']
alice said to